In [ ]:
from datasets import load_dataset
dataset = load_dataset("keremberke/chest-xray-classification",'full')
dataset.save_to_disk("./chest_xray_clasnsificatio")

In [ ]:
import torch
import accelerate
from torch.utils.data import DataLoader
from torchvision.transforms import (
    Compose, Resize, ToTensor, Normalize
)
from transformers import ViTConfig, ViTForImageClassification, Trainer, TrainingArguments
from datasets import DatasetDict

# Load the dataset
dataset = DatasetDict.load_from_disk("./chest_xray_clasnsificatio")

image_size = 224
# Define a sequence of transformations for preprocessing images, including resizing, 
# converting to tensor, and normalizing pixel values
transforms = Compose([
    Resize((image_size, image_size)),
    ToTensor(),
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Function to preprocess data by applying transformations to images
def preprocess_data(example):
    example['pixel_values'] = transforms(example['image'])
    return example

# Apply the preprocessing function to all samples in the dataset
dataset = dataset.map(preprocess_data, batched=False)

# Remove unnecessary columns from the dataset
dataset = dataset.remove_columns(["image_file_path", "image"])

# Define a Vision Transformer (ViT) configuration with specific parameters
config = ViTConfig(
    image_size=image_size,
    num_channels=3,
    patch_size=16,
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    intermediate_size=3072,
    num_labels=2
)

# Instantiate a Vision Transformer model for image classification using the specified configuration
model = ViTForImageClassification(config)

# Define training arguments for the Trainer, including batch size, epochs, and logging options
training_args = TrainingArguments(
    output_dir="./vit_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False
)

from evaluate import load

# Load the 'accuracy' metric from the evaluation library
metric = load("accuracy")

# Function to compute evaluation metrics during training and validation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return metric.compute(predictions=predictions.numpy(), references=labels)

# Create a Trainer object for training and evaluating the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=None,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model on the test dataset
results = trainer.evaluate(dataset["test"])
print("Test Results:", results)


C:\Users\19811\AppData\Local\Temp\ipykernel_8308\1335099057.py:75: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\19811\_netrc


  0%|          | 0/2550 [00:00<?, ?it/s]

{'loss': 1.2936, 'grad_norm': 23.22121238708496, 'learning_rate': 4.980392156862745e-05, 'epoch': 0.04}
{'loss': 0.7692, 'grad_norm': 0.4044587314128876, 'learning_rate': 4.960784313725491e-05, 'epoch': 0.08}
{'loss': 0.5978, 'grad_norm': 2.999021053314209, 'learning_rate': 4.9411764705882355e-05, 'epoch': 0.12}
{'loss': 0.5879, 'grad_norm': 2.198652982711792, 'learning_rate': 4.9215686274509804e-05, 'epoch': 0.16}
{'loss': 0.6272, 'grad_norm': 1.4779688119888306, 'learning_rate': 4.901960784313725e-05, 'epoch': 0.2}
{'loss': 0.5429, 'grad_norm': 5.54686164855957, 'learning_rate': 4.882352941176471e-05, 'epoch': 0.24}
{'loss': 0.5327, 'grad_norm': 7.827354431152344, 'learning_rate': 4.862745098039216e-05, 'epoch': 0.27}
{'loss': 0.6011, 'grad_norm': 8.274473190307617, 'learning_rate': 4.843137254901961e-05, 'epoch': 0.31}
{'loss': 0.6586, 'grad_norm': 9.161334037780762, 'learning_rate': 4.823529411764706e-05, 'epoch': 0.35}
{'loss': 0.8176, 'grad_norm': 5.5970377922058105, 'learning_ra

  0%|          | 0/73 [00:00<?, ?it/s]

{'eval_loss': 0.2792075276374817, 'eval_accuracy': 0.878969957081545, 'eval_runtime': 233.7612, 'eval_samples_per_second': 4.984, 'eval_steps_per_second': 0.312, 'epoch': 1.0}
{'loss': 0.2869, 'grad_norm': 15.761570930480957, 'learning_rate': 4.490196078431373e-05, 'epoch': 1.02}
{'loss': 0.4262, 'grad_norm': 15.251462936401367, 'learning_rate': 4.470588235294118e-05, 'epoch': 1.06}
{'loss': 0.4244, 'grad_norm': 14.319478034973145, 'learning_rate': 4.450980392156863e-05, 'epoch': 1.1}
{'loss': 0.3106, 'grad_norm': 8.241303443908691, 'learning_rate': 4.431372549019608e-05, 'epoch': 1.14}
{'loss': 0.3482, 'grad_norm': 11.514594078063965, 'learning_rate': 4.411764705882353e-05, 'epoch': 1.18}
{'loss': 0.5182, 'grad_norm': 15.224875450134277, 'learning_rate': 4.392156862745098e-05, 'epoch': 1.22}
{'loss': 0.3725, 'grad_norm': 1.137607455253601, 'learning_rate': 4.3725490196078437e-05, 'epoch': 1.25}
{'loss': 0.279, 'grad_norm': 3.010125160217285, 'learning_rate': 4.3529411764705885e-05, 'e

...

{'eval_loss': 0.20923149585723877, 'eval_accuracy': 0.9407725321888412, 'eval_runtime': 150.5163, 'eval_samples_per_second': 7.74, 'eval_steps_per_second': 0.485, 'epoch': 9.0}
{'loss': 0.1528, 'grad_norm': 0.14838454127311707, 'learning_rate': 4.901960784313726e-06, 'epoch': 9.02}
{'loss': 0.0413, 'grad_norm': 15.815964698791504, 'learning_rate': 4.705882352941177e-06, 'epoch': 9.06}
{'loss': 0.1437, 'grad_norm': 7.116632461547852, 'learning_rate': 4.509803921568627e-06, 'epoch': 9.1}
{'loss': 0.1142, 'grad_norm': 28.419443130493164, 'learning_rate': 4.313725490196079e-06, 'epoch': 9.14}
{'loss': 0.0192, 'grad_norm': 0.368515282869339, 'learning_rate': 4.11764705882353e-06, 'epoch': 9.18}
{'loss': 0.0649, 'grad_norm': 0.19300584495067596, 'learning_rate': 3.92156862745098e-06, 'epoch': 9.22}
{'loss': 0.1388, 'grad_norm': 2.903005838394165, 'learning_rate': 3.7254901960784316e-06, 'epoch': 9.25}
{'loss': 0.096, 'grad_norm': 14.219625473022461, 'learning_rate': 3.5294117647058825e-06, '

  0%|          | 0/73 [00:00<?, ?it/s]

{'eval_loss': 0.20011577010154724, 'eval_accuracy': 0.944206008583691, 'eval_runtime': 208.2464, 'eval_samples_per_second': 5.594, 'eval_steps_per_second': 0.351, 'epoch': 10.0}
{'train_runtime': 14666.295, 'train_samples_per_second': 2.78, 'train_steps_per_second': 0.174, 'train_loss': 0.24141756545094883, 'epoch': 10.0}


  0%|          | 0/37 [00:00<?, ?it/s]

Test Results: {'eval_loss': 0.23524129390716553, 'eval_accuracy': 0.9415807560137457, 'eval_runtime': 90.1285, 'eval_samples_per_second': 6.457, 'eval_steps_per_second': 0.411, 'epoch': 10.0}


In [ ]:
#save the model
model.save_pretrained("./vit_model")
from transformers import ViTForImageClassification

loaded_model = ViTForImageClassification.from_pretrained("./vit_model")

print(loaded_model)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe